In [2]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

In [3]:
path = "../CP_analysis/"

In [4]:
sessions_raw = pd.read_csv(path + "sessions.csv", index_col=False)
# sessions = pd.read_csv(path + "sessions.csv", index_col=False, nrows=1e6)
sessions_raw.head(1)

,Session ID,Station ID,Port ID,Connector Type,POI Category,POI Subcategory,Station Start Time (Local),Station End Time (Local),Session Time Zone,Session Time (secs),...,City,County,State,Country,Driver City,Driver State,Driver Country,Max Power,Session Type,Vehicle Model Year
0,99999951,155805,208243.0,J1772,Parking,Commercial,2017-09-21 13:50:12.0,2017-09-21 14:59:12.0,PDT,4143,...,Santa Clara,Santa Clara County,California,United States,San Jose,California,United States,6.6,OTHER,2015.0


In [5]:
# print(len(sessions))
# for column in list(sessions):
#     unique = sessions[column].unique()
#     print(column, len(unique))
#     if len(unique) < 25:
#         print(unique)

In [6]:
sessions = sessions_raw.copy(deep=True)
sessions = sessions[sessions['Session Type'] != 'FLEET']
sessions = sessions[sessions["POI Category"] == "Workplace"]

In [7]:
sessions['slack_time'] = sessions['Session Time (secs)'] - sessions['Charging Time (secs)']

sessions['year'] = sessions["Station Start Time (Local)"].apply(lambda x: int(x[:4]))
sessions = sessions[sessions['year'].apply(lambda x: x in [2016, 2017, 2018])]

In [8]:
# col = 'Energy (kWh)'
# df = sessions.copy(deep=True)

# df = df[df[col] < np.percentile(df[col], 99)]
# df = df[df[col] > 0]
# plot = plt.hist(df[col], bins=50)

In [9]:
# find list of suitable stations
stations = sessions.copy(deep=True)
stations = stations[stations['Energy (kWh)'] > 5]
stations = stations[stations['slack_time'] > 7200]
counts = stations['Port ID'].value_counts()
print(len(stations))
print(len(counts))
print(counts.head(10))

1673743
10775
153983.0    845
211483.0    776
154489.0    765
401831.0    742
153985.0    693
139477.0    689
211481.0    681
154491.0    669
112645.0    631
162851.0    624
Name: Port ID, dtype: int64


In [10]:
ports_1000 = counts.head(1000)
ports_1000 = list(ports_1000.index)

In [11]:
sessions_1000 = sessions[sessions['Port ID'].apply(lambda x: x in ports_1000)]

In [12]:
counts = sessions_1000['City'].value_counts()
print(len(counts))
print(counts.head(10))

25
Mountain View    455754
Cupertino        243591
Sunnyvale        172559
Palo Alto         67285
Los Gatos         56191
Santa Clara       44903
San Jose          34895
San Bruno         26408
Menlo Park        22948
Emeryville        18340
Name: City, dtype: int64


In [13]:
df = sessions_1000.copy(deep=True)
df = df.groupby(['Zip Code', 'Port ID'], as_index=False)
counts = df['Session ID'].agg('count').sort_values(['Session ID'])
print(len(counts))
print(counts.tail(10))

1002
     Zip Code   Port ID  Session ID
821     95014  165995.0        2491
791     95014  145255.0        2531
872     95014  429681.0        2583
819     95014  165281.0        2599
860     95014  199775.0        2622
820     95014  165283.0        2657
878     95014  779671.0        2685
730     94710  797351.0        2752
861     95014  204347.0        3892
862     95014  204349.0        3992


In [17]:
sessions_95014 = sessions_1000.copy(deep=True)
sessions_95014 = sessions_95014[sessions_95014['Zip Code'] == 95014]
counts = sessions_95014['Port ID'].value_counts()

print(len(counts))
print(counts.head(10))
print(counts.tail(10))

ports_95014_top10 = counts.head(10)
ports_95014_top10 = list(ports_95014_top10.index)
ports_95014_tail10 = counts.tail(10)
ports_95014_tail10 = list(ports_95014_tail10.index)

145
204349.0    3992
204347.0    3892
779671.0    2685
165283.0    2657
199775.0    2622
165281.0    2599
429681.0    2583
145255.0    2531
165995.0    2491
401831.0    2464
Name: Port ID, dtype: int64
794761.0    1119
121533.0    1111
162343.0    1092
197543.0    1088
222643.0    1078
222645.0    1066
159697.0    1062
162345.0    1055
123291.0    1018
197737.0     988
Name: Port ID, dtype: int64


In [18]:
sessions_95014_top10 = sessions_95014[sessions_95014['Port ID'].apply(lambda x: x in ports_95014_top10)]
sessions_95014_tail10 = sessions_95014[sessions_95014['Port ID'].apply(lambda x: x in ports_95014_tail10)]
print(len(sessions_95014_top10))
print(len(sessions_95014_tail10))


28516
10677


In [19]:
keep_cols = ["Port ID", "Energy (kWh)", "Station Start Time (Local)", "Session Time (secs)"]
# keep_cols = keep_cols + ["Driver ID", "Battery Capacity", "City", "County"]

In [20]:
sessions_1000[keep_cols].to_csv(path + "sessions_161718_top1000.csv", index=False)
sessions_95014_top10[keep_cols].to_csv(path + "sessions_161718_95014_top10.csv", index=False)
sessions_95014_tail10[keep_cols].to_csv(path + "sessions_161718_95014_tail10.csv", index=False)